In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# !pip install gensim

In [3]:
from nltk import sent_tokenize,word_tokenize
from gensim.utils import simple_preprocess
import gensim

In [4]:
from gensim.models import Word2Vec, KeyedVectors

In [5]:
import gensim.downloader as api

In [6]:
wv = api.load('word2vec-google-news-300')
vec_king = wv['king']
vec_king

array([ 1.25976562e-01,  2.97851562e-02,  8.60595703e-03,  1.39648438e-01,
       -2.56347656e-02, -3.61328125e-02,  1.11816406e-01, -1.98242188e-01,
        5.12695312e-02,  3.63281250e-01, -2.42187500e-01, -3.02734375e-01,
       -1.77734375e-01, -2.49023438e-02, -1.67968750e-01, -1.69921875e-01,
        3.46679688e-02,  5.21850586e-03,  4.63867188e-02,  1.28906250e-01,
        1.36718750e-01,  1.12792969e-01,  5.95703125e-02,  1.36718750e-01,
        1.01074219e-01, -1.76757812e-01, -2.51953125e-01,  5.98144531e-02,
        3.41796875e-01, -3.11279297e-02,  1.04492188e-01,  6.17675781e-02,
        1.24511719e-01,  4.00390625e-01, -3.22265625e-01,  8.39843750e-02,
        3.90625000e-02,  5.85937500e-03,  7.03125000e-02,  1.72851562e-01,
        1.38671875e-01, -2.31445312e-01,  2.83203125e-01,  1.42578125e-01,
        3.41796875e-01, -2.39257812e-02, -1.09863281e-01,  3.32031250e-02,
       -5.46875000e-02,  1.53198242e-02, -1.62109375e-01,  1.58203125e-01,
       -2.59765625e-01,  

# training word to vec from scrach

## importing data

In [7]:
# import pandas as pd
messages = pd.read_csv("C://Users//prach//OneDrive//Desktop//NOTES ML And DL//Deep Learning//2_NLP_spam_ham_project_word2vec//SMSSpamCollection.txt",sep= '\t',names=['label','message'] )

In [8]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [9]:
messages.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
8,spam,WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030


In [10]:
messages['label'].value_counts()

label
ham     4825
spam     747
Name: count, dtype: int64

## data cleaning and preprocessing

In [11]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prach\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prach\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [14]:
len(messages)

5572

In [15]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\prach\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [16]:
def get_wordnet_pos(word):
    """Convert POS tag from nltk.pos_tag to WordNet format"""
    tag = nltk.pos_tag([word])[0][1][0].upper()  # Get first letter of POS tag
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [17]:
from nltk.corpus import wordnet


In [18]:
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ', messages['message'][i])
    review= review.lower()
    word_token = word_tokenize(review)
    word_pos= nltk.pos_tag(word_token)
    review = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word, tag in word_pos]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['go until jurong point crazy available only in bugis n great world la e buffet cine there get amore wat',
 'ok lar joking wif u oni',
 'free entry in a wkly comp to win fa cup final tkts st may text fa to to receive entry question std txt rate t c s apply over s',
 'u dun say so early hor u c already then say',
 'nah i don t think he go to usf he life around here though',
 'freemsg hey there darling it s be week s now and no word back i d like some fun you up for it still tb ok xxx std chgs to send to rcv',
 'even my brother be not like to speak with me they treat me like aid patent',
 'a per your request melle melle oru minnaminunginte nurungu vettam have be set a your callertune for all caller press to copy your friend callertune',
 'winner a a value network customer you have be select to receivea prize reward to claim call claim code kl valid hour only',
 'have your mobile month or more u r entitle to update to the late colour mobile with camera for free call the mobile update co f

In [20]:
len(corpus)

5572

In [21]:
# [for i,j, k in zip(list(map(len,corpus)), corpus, messages['message']) if i<1]
[[i,j,k]for i, j, k in zip(list(map(len, corpus)), corpus, messages['message']) if i < 1]


[[0, '', '645'], [0, '', ':) '], [0, '', ':-) :-)']]

In [22]:
words =[]
for sent in corpus:
    sent_token= sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [23]:
# import nltk
# nltk.download('punkt')

In [24]:
words

[['go',
  'until',
  'jurong',
  'point',
  'crazy',
  'available',
  'only',
  'in',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'there',
  'get',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'in',
  'wkly',
  'comp',
  'to',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'to',
  'to',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply',
  'over'],
 ['dun', 'say', 'so', 'early', 'hor', 'already', 'then', 'say'],
 ['nah',
  'don',
  'think',
  'he',
  'go',
  'to',
  'usf',
  'he',
  'life',
  'around',
  'here',
  'though'],
 ['freemsg',
  'hey',
  'there',
  'darling',
  'it',
  'be',
  'week',
  'now',
  'and',
  'no',
  'word',
  'back',
  'like',
  'some',
  'fun',
  'you',
  'up',
  'for',
  'it',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'to',
  'send',
  'to',
  'rcv'],
 ['even',
  'my',
  'brother',
  'be',
  'not',
  'like',
  'to',
  'speak'

## train word2vec

In [25]:
import gensim
model = gensim.models.Word2Vec(words, vector_size = 100)

In [26]:
model.wv.index_to_key

['be',
 'to',
 'you',
 'the',
 'and',
 'it',
 'in',
 'have',
 'me',
 'my',
 'do',
 'get',
 'for',
 'call',
 'your',
 'of',
 'that',
 'go',
 'on',
 'now',
 'can',
 'so',
 'not',
 'but',
 'or',
 'we',
 'at',
 'ur',
 'if',
 'will',
 'with',
 'no',
 'just',
 'this',
 'come',
 'gt',
 'lt',
 'how',
 'up',
 'when',
 'ok',
 'day',
 'what',
 'free',
 'from',
 'all',
 'know',
 'out',
 'll',
 'like',
 'love',
 'want',
 'time',
 'good',
 'then',
 'there',
 'he',
 'say',
 'text',
 'only',
 'send',
 'take',
 'need',
 'see',
 'one',
 'txt',
 'today',
 'make',
 'by',
 'stop',
 'don',
 'she',
 'home',
 'about',
 'think',
 'reply',
 'well',
 'lor',
 'sorry',
 'tell',
 'still',
 'mobile',
 'back',
 'da',
 'dont',
 'our',
 'phone',
 'week',
 'hi',
 'new',
 'work',
 'they',
 'please',
 'later',
 'miss',
 'any',
 'pls',
 'her',
 'co',
 'msg',
 'give',
 'min',
 'ask',
 'an',
 'some',
 'dear',
 'message',
 'night',
 'who',
 'here',
 'wait',
 'where',
 're',
 'thing',
 'try',
 'great',
 'hope',
 'much',
 'oh',

In [27]:
len(model.wv.index_to_key)

1634

In [28]:
model.corpus_count

5569

In [29]:
model.epochs

5

In [30]:
model.wv.similar_by_word('kid')

[('hey', 0.997113049030304),
 ('something', 0.9970312714576721),
 ('many', 0.9970086216926575),
 ('also', 0.9969524145126343),
 ('thought', 0.9969442486763),
 ('wish', 0.996938943862915),
 ('car', 0.99689120054245),
 ('even', 0.9968789219856262),
 ('eat', 0.9968779683113098),
 ('life', 0.9968525171279907)]

In [31]:
model.wv['good'].shape

(100,)

In [32]:
words

[['go',
  'until',
  'jurong',
  'point',
  'crazy',
  'available',
  'only',
  'in',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'there',
  'get',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'in',
  'wkly',
  'comp',
  'to',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'to',
  'to',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply',
  'over'],
 ['dun', 'say', 'so', 'early', 'hor', 'already', 'then', 'say'],
 ['nah',
  'don',
  'think',
  'he',
  'go',
  'to',
  'usf',
  'he',
  'life',
  'around',
  'here',
  'though'],
 ['freemsg',
  'hey',
  'there',
  'darling',
  'it',
  'be',
  'week',
  'now',
  'and',
  'no',
  'word',
  'back',
  'like',
  'some',
  'fun',
  'you',
  'up',
  'for',
  'it',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'to',
  'send',
  'to',
  'rcv'],
 ['even',
  'my',
  'brother',
  'be',
  'not',
  'like',
  'to',
  'speak'

In [33]:
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0)
    

In [34]:
# !pip install tqdm

In [35]:
from tqdm import tqdm
import numpy as np

In [36]:
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

  0%|                                                                                         | 0/5569 [00:00<?, ?it/s]C:\Users\prach\miniconda3\envs\numpyenv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\prach\miniconda3\envs\numpyenv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|███████████████████████████████████████████████████████████████████████████| 5569/5569 [00:00<00:00, 11625.01it/s]


In [37]:
len(X)

5569

In [38]:
X

[array([-0.08301246,  0.23366874,  0.02245077,  0.07034831,  0.03303981,
        -0.34047112,  0.07549433,  0.67209345, -0.28359184, -0.21641897,
        -0.10889189, -0.44878018,  0.00750134,  0.16882752,  0.10279118,
        -0.18684642,  0.10348218, -0.21364132, -0.02782989, -0.60293615,
         0.20037594,  0.11055587,  0.09134833, -0.22553161, -0.080695  ,
         0.00981103, -0.2867045 , -0.15117072, -0.23727383,  0.09134663,
         0.26685077, -0.01869288,  0.1431    , -0.12256381, -0.1367201 ,
         0.28887632,  0.0927046 , -0.25187692, -0.23971014, -0.5057075 ,
         0.03341748, -0.20672414, -0.09288166,  0.01532945,  0.26403913,
        -0.16046578, -0.12645386, -0.09311235,  0.1462911 ,  0.05944   ,
         0.13795802, -0.19474502, -0.00187726, -0.04231008, -0.0349557 ,
         0.04362673,  0.16849466, -0.10813646, -0.26968986,  0.08507977,
        -0.02046985,  0.00963173,  0.04497721, -0.02075845, -0.30341968,
         0.28022337,  0.16285497,  0.12641354, -0.4

#### inpedendent feature

In [39]:
# X_new = np.array(X)

In [40]:
shapes = [arr.shape for arr in X]
print(set(shapes))

{(), (100,)}


In [41]:
shapes = [arr for arr in X if arr.shape == ()]
print(set(shapes))

{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan}


In [42]:
X_filled = [arr if arr.shape == (100,) else np.zeros(100) for arr in X]

In [43]:
X_new = np.vstack(X_filled)  # convrting to 2d

In [44]:
X_new

array([[-0.08301246,  0.23366874,  0.02245077, ..., -0.13311712,
         0.11741015, -0.04939111],
       [-0.08130829,  0.2054674 ,  0.01233918, ..., -0.12747966,
         0.10575435, -0.03161952],
       [-0.13446213,  0.25304416,  0.04128958, ..., -0.13398479,
         0.09172973, -0.0156072 ],
       ...,
       [-0.08434655,  0.30161372,  0.02552666, ..., -0.17104065,
         0.15825558, -0.08187811],
       [-0.09310603,  0.27511266,  0.02494917, ..., -0.15527973,
         0.13600421, -0.06238621],
       [-0.11114483,  0.29251245,  0.02449133, ..., -0.18150875,
         0.15161893, -0.06245689]])

In [45]:
X_new[0]

array([-0.08301246,  0.23366874,  0.02245077,  0.07034831,  0.03303981,
       -0.34047112,  0.07549433,  0.67209345, -0.28359184, -0.21641897,
       -0.10889189, -0.44878018,  0.00750134,  0.16882752,  0.10279118,
       -0.18684642,  0.10348218, -0.21364132, -0.02782989, -0.60293615,
        0.20037594,  0.11055587,  0.09134833, -0.22553161, -0.080695  ,
        0.00981103, -0.28670451, -0.15117072, -0.23727383,  0.09134663,
        0.26685077, -0.01869288,  0.14309999, -0.12256381, -0.13672011,
        0.28887632,  0.0927046 , -0.25187692, -0.23971014, -0.5057075 ,
        0.03341748, -0.20672414, -0.09288166,  0.01532945,  0.26403913,
       -0.16046578, -0.12645386, -0.09311235,  0.14629111,  0.05944   ,
        0.13795802, -0.19474502, -0.00187726, -0.04231008, -0.0349557 ,
        0.04362673,  0.16849466, -0.10813646, -0.26968986,  0.08507977,
       -0.02046985,  0.00963173,  0.04497721, -0.02075845, -0.30341968,
        0.28022337,  0.16285497,  0.12641354, -0.402852  ,  0.35

In [46]:
X_new[0].shape

(100,)

In [47]:
len(X_new)

5569

#### dependent feature

In [48]:
# [[i,j,k]for i, j, k in zip(list(map(len, corpus)), corpus, messages['message']) if i < 1]  ->to remove these data
y=messages[list(map(lambda x: len(x)>0 , corpus))]

In [49]:
y= pd.get_dummies(y['label'])

In [50]:
y = y.iloc[:,0].values

In [51]:
y

array([ True,  True, False, ...,  True,  True,  True])

In [52]:
len(y)

5569

In [53]:
y.shape

(5569,)

#### converting independent and dep var in dataframe

In [54]:
# df= pd.DataFrame()
# for i in range(len(X)):
#     df = df.append(pd.DataFrame(X_new[i].reshape(1,-1)))
df = pd.concat([pd.DataFrame(X_new[i].reshape(1, -1)) for i in range(len(X))], ignore_index=True)

In [55]:
df.shape

(5569, 100)

In [56]:
df['Output'] = y

##### drop null values

In [57]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,Output
0,-0.083012,0.233669,0.022451,0.070348,0.033040,-0.340471,0.075494,0.672093,-0.283592,-0.216419,-0.108892,-0.448780,0.007501,0.168828,0.102791,-0.186846,0.103482,-0.213641,-0.027830,-0.602936,0.200376,0.110556,0.091348,-0.225532,-0.080695,0.009811,-0.286705,-0.151171,-0.237274,0.091347,0.266851,-0.018693,0.143100,-0.122564,-0.136720,0.288876,0.092705,-0.251877,-0.239710,-0.505708,0.033417,-0.206724,-0.092882,0.015329,0.264039,-0.160466,-0.126454,-0.093112,0.146291,0.059440,0.137958,-0.194745,-0.001877,-0.042310,-0.034956,0.043627,0.168495,-0.108136,-0.269690,0.085080,-0.020470,0.009632,0.044977,-0.020758,-0.303420,0.280223,0.162855,0.126414,-0.402852,0.353242,-0.201266,0.055935,0.382608,-0.046151,0.268719,0.140245,0.014305,-0.092722,-0.227004,-0.051976,-0.149137,-0.010712,-0.339865,0.339461,-0.039607,0.035756,0.103425,0.178057,0.341291,0.094742,0.281759,0.221229,0.064312,0.024156,0.422750,0.245720,0.191908,-0.133117,0.117410,-0.049391,True
1,-0.081308,0.205467,0.012339,0.052701,0.038623,-0.301440,0.060275,0.587610,-0.241245,-0.193713,-0.106752,-0.397245,0.008865,0.151816,0.088971,-0.165402,0.089827,-0.198499,-0.025132,-0.536628,0.183720,0.096632,0.082748,-0.197713,-0.067894,0.013641,-0.258935,-0.132560,-0.204940,0.092797,0.245118,-0.016615,0.120742,-0.098853,-0.126043,0.256695,0.081825,-0.229696,-0.201918,-0.443346,0.037403,-0.181811,-0.093919,0.002587,0.216608,-0.143915,-0.106497,-0.076190,0.127769,0.040592,0.125976,-0.178525,0.005551,-0.017363,-0.040003,0.049657,0.149803,-0.098299,-0.248087,0.073519,-0.018655,0.004774,0.042850,-0.011538,-0.255897,0.241002,0.140254,0.107382,-0.344080,0.319856,-0.185150,0.050404,0.341530,-0.051968,0.237719,0.124360,-0.000997,-0.073583,-0.199148,-0.053224,-0.126670,-0.005842,-0.299230,0.296949,-0.037357,0.031697,0.091046,0.168405,0.286225,0.078536,0.263840,0.188747,0.056113,0.007903,0.374924,0.221299,0.161776,-0.127480,0.105754,-0.031620,True
2,-0.134462,0.253044,0.041290,0.080743,0.053588,-0.362703,0.073922,0.733257,-0.272003,-0.259690,-0.137188,-0.480222,-0.008870,0.182584,0.091987,-0.180521,0.111676,-0.256055,-0.031775,-0.644913,0.194646,0.158881,0.089974,-0.238877,-0.059527,0.040374,-0.341575,-0.135872,-0.260633,0.087794,0.241260,-0.011441,0.134366,-0.088765,-0.150098,0.279130,0.097360,-0.258300,-0.211780,-0.545704,0.033375,-0.235535,-0.086438,0.028839,0.307675,-0.170513,-0.135141,-0.056684,0.150748,0.064947,0.142616,-0.206611,-0.001596,-0.081485,-0.030698,0.073371,0.221275,-0.108854,-0.290772,0.126918,-0.036330,0.014540,0.050204,-0.010140,-0.311745,0.300758,0.192485,0.147718,-0.407058,0.383958,-0.204763,0.035026,0.425656,-0.048118,0.278466,0.111676,-0.021807,-0.095062,-0.229167,-0.067758,-0.133468,0.024283,-0.314189,0.372889,0.012658,0.054938,0.129167,0.181531,0.391166,0.091822,0.361442,0.252149,0.056664,0.062673,0.432143,0.260767,0.191473,-0.133985,0.091730,-0.015607,False
3,-0.109986,0.319041,0.013460,0.091999,0.048233,-0.470771,0.090374,0.908940,-0.396872,-0.285608,-0.150484,-0.621303,0.019225,0.227041,0.147179,-0.255671,0.154608,-0.289697,-0.052208,-0.827037,0.275147,0.124350,0.130671,-0.295393,-0.115990,0.003202,-0.381685,-0.208303,-0.304687,0.140537,0.394790,-0.040573,0.192122,-0.166500,-0.181173,0.402168,0.134967,-0.367205,-0.326594,-0.679763,0.056695,-0.278717,-0.141231,0.012855,0.333476,-0.215075,-0.168285,-0.150031,0.193257,0.063506,0.194929,-0.279013,0.016530,-0.004745,-0.079164,0.058414,0.209117,-0.156847,-0.383591,0.102354,-0.014476,0.000519,0.064139,-0.023825,-0.412322,0.376637,0.213193,0.160999,-0.547795,0.488083,-0.291491,0.095671,0.515340,-0.084387,0.360959,0.216115,0.024445,-0.120463,-0.320413,-0.074403,-0.204968,-0.031394,-0.477954,0.445324,-0.083487,0.051057,0.128540,0.247082,0

In [58]:
df.columns

Index([       0,        1,        2,        3,        4,        5,        6,
              7,        8,        9,
       ...
             91,       92,       93,       94,       95,       96,       97,
             98,       99, 'Output'],
      dtype='object', length=101)

In [59]:
df= df[df[0]!=0]

In [60]:
X=df.drop(columns=['Output'])

In [61]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,-0.083012,0.233669,0.022451,0.070348,0.033040,-0.340471,0.075494,0.672093,-0.283592,-0.216419,-0.108892,-0.448780,0.007501,0.168828,0.102791,-0.186846,0.103482,-0.213641,-0.027830,-0.602936,0.200376,0.110556,0.091348,-0.225532,-0.080695,0.009811,-0.286705,-0.151171,-0.237274,0.091347,0.266851,-0.018693,0.143100,-0.122564,-0.136720,0.288876,0.092705,-0.251877,-0.239710,-0.505708,0.033417,-0.206724,-0.092882,0.015329,0.264039,-0.160466,-0.126454,-0.093112,0.146291,0.059440,0.137958,-0.194745,-0.001877,-0.042310,-0.034956,0.043627,0.168495,-0.108136,-0.269690,0.085080,-0.020470,0.009632,0.044977,-0.020758,-0.303420,0.280223,0.162855,0.126414,-0.402852,0.353242,-0.201266,0.055935,0.382608,-0.046151,0.268719,0.140245,0.014305,-0.092722,-0.227004,-0.051976,-0.149137,-0.010712,-0.339865,0.339461,-0.039607,0.035756,0.103425,0.178057,0.341291,0.094742,0.281759,0.221229,0.064312,0.024156,0.422750,0.245720,0.191908,-0.133117,0.117410,-0.049391
1,-0.081308,0.205467,0.012339,0.052701,0.038623,-0.301440,0.060275,0.587610,-0.241245,-0.193713,-0.106752,-0.397245,0.008865,0.151816,0.088971,-0.165402,0.089827,-0.198499,-0.025132,-0.536628,0.183720,0.096632,0.082748,-0.197713,-0.067894,0.013641,-0.258935,-0.132560,-0.204940,0.092797,0.245118,-0.016615,0.120742,-0.098853,-0.126043,0.256695,0.081825,-0.229696,-0.201918,-0.443346,0.037403,-0.181811,-0.093919,0.002587,0.216608,-0.143915,-0.106497,-0.076190,0.127769,0.040592,0.125976,-0.178525,0.005551,-0.017363,-0.040003,0.049657,0.149803,-0.098299,-0.248087,0.073519,-0.018655,0.004774,0.042850,-0.011538,-0.255897,0.241002,0.140254,0.107382,-0.344080,0.319856,-0.185150,0.050404,0.341530,-0.051968,0.237719,0.124360,-0.000997,-0.073583,-0.199148,-0.053224,-0.126670,-0.005842,-0.299230,0.296949,-0.037357,0.031697,0.091046,0.168405,0.286225,0.078536,0.263840,0.188747,0.056113,0.007903,0.374924,0.221299,0.161776,-0.127480,0.105754,-0.031620
2,-0.134462,0.253044,0.041290,0.080743,0.053588,-0.362703,0.073922,0.733257,-0.272003,-0.259690,-0.137188,-0.480222,-0.008870,0.182584,0.091987,-0.180521,0.111676,-0.256055,-0.031775,-0.644913,0.194646,0.158881,0.089974,-0.238877,-0.059527,0.040374,-0.341575,-0.135872,-0.260633,0.087794,0.241260,-0.011441,0.134366,-0.088765,-0.150098,0.279130,0.097360,-0.258300,-0.211780,-0.545704,0.033375,-0.235535,-0.086438,0.028839,0.307675,-0.170513,-0.135141,-0.056684,0.150748,0.064947,0.142616,-0.206611,-0.001596,-0.081485,-0.030698,0.073371,0.221275,-0.108854,-0.290772,0.126918,-0.036330,0.014540,0.050204,-0.010140,-0.311745,0.300758,0.192485,0.147718,-0.407058,0.383958,-0.204763,0.035026,0.425656,-0.048118,0.278466,0.111676,-0.021807,-0.095062,-0.229167,-0.067758,-0.133468,0.024283,-0.314189,0.372889,0.012658,0.054938,0.129167,0.181531,0.391166,0.091822,0.361442,0.252149,0.056664,0.062673,0.432143,0.260767,0.191473,-0.133985,0.091730,-0.015607
3,-0.109986,0.319041,0.013460,0.091999,0.048233,-0.470771,0.090374,0.908940,-0.396872,-0.285608,-0.150484,-0.621303,0.019225,0.227041,0.147179,-0.255671,0.154608,-0.289697,-0.052208,-0.827037,0.275147,0.124350,0.130671,-0.295393,-0.115990,0.003202,-0.381685,-0.208303,-0.304687,0.140537,0.394790,-0.040573,0.192122,-0.166500,-0.181173,0.402168,0.134967,-0.367205,-0.326594,-0.679763,0.056695,-0.278717,-0.141231,0.012855,0.333476,-0.215075,-0.168285,-0.150031,0.193257,0.063506,0.194929,-0.279013,0.016530,-0.004745,-0.079164,0.058414,0.209117,-0.156847,-0.383591,0.102354,-0.014476,0.000519,0.064139,-0.023825,-0.412322,0.376637,0.213193,0.160999,-0.547795,0.488083,-0.291491,0.095671,0.515340,-0.084387,0.360959,0.216115,0.024445,-0.120463,-0.320413,-0.074403,-0.204968,-0.031394,-0.477954,0.445324,-0.083487,0.051057,0.128540,0.247082,0.423910,0.124251,0.3736

In [62]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5558 entries, 0 to 5568
Data columns (total 100 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5558 non-null   float64
 1   1       5558 non-null   float64
 2   2       5558 non-null   float64
 3   3       5558 non-null   float64
 4   4       5558 non-null   float64
 5   5       5558 non-null   float64
 6   6       5558 non-null   float64
 7   7       5558 non-null   float64
 8   8       5558 non-null   float64
 9   9       5558 non-null   float64
 10  10      5558 non-null   float64
 11  11      5558 non-null   float64
 12  12      5558 non-null   float64
 13  13      5558 non-null   float64
 14  14      5558 non-null   float64
 15  15      5558 non-null   float64
 16  16      5558 non-null   float64
 17  17      5558 non-null   float64
 18  18      5558 non-null   float64
 19  19      5558 non-null   float64
 20  20      5558 non-null   float64
 21  21      5558 non-null   float64
 22  22  

In [63]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,Output
0,-0.083012,0.233669,0.022451,0.070348,0.033040,-0.340471,0.075494,0.672093,-0.283592,-0.216419,-0.108892,-0.448780,0.007501,0.168828,0.102791,-0.186846,0.103482,-0.213641,-0.027830,-0.602936,0.200376,0.110556,0.091348,-0.225532,-0.080695,0.009811,-0.286705,-0.151171,-0.237274,0.091347,0.266851,-0.018693,0.143100,-0.122564,-0.136720,0.288876,0.092705,-0.251877,-0.239710,-0.505708,0.033417,-0.206724,-0.092882,0.015329,0.264039,-0.160466,-0.126454,-0.093112,0.146291,0.059440,0.137958,-0.194745,-0.001877,-0.042310,-0.034956,0.043627,0.168495,-0.108136,-0.269690,0.085080,-0.020470,0.009632,0.044977,-0.020758,-0.303420,0.280223,0.162855,0.126414,-0.402852,0.353242,-0.201266,0.055935,0.382608,-0.046151,0.268719,0.140245,0.014305,-0.092722,-0.227004,-0.051976,-0.149137,-0.010712,-0.339865,0.339461,-0.039607,0.035756,0.103425,0.178057,0.341291,0.094742,0.281759,0.221229,0.064312,0.024156,0.422750,0.245720,0.191908,-0.133117,0.117410,-0.049391,True
1,-0.081308,0.205467,0.012339,0.052701,0.038623,-0.301440,0.060275,0.587610,-0.241245,-0.193713,-0.106752,-0.397245,0.008865,0.151816,0.088971,-0.165402,0.089827,-0.198499,-0.025132,-0.536628,0.183720,0.096632,0.082748,-0.197713,-0.067894,0.013641,-0.258935,-0.132560,-0.204940,0.092797,0.245118,-0.016615,0.120742,-0.098853,-0.126043,0.256695,0.081825,-0.229696,-0.201918,-0.443346,0.037403,-0.181811,-0.093919,0.002587,0.216608,-0.143915,-0.106497,-0.076190,0.127769,0.040592,0.125976,-0.178525,0.005551,-0.017363,-0.040003,0.049657,0.149803,-0.098299,-0.248087,0.073519,-0.018655,0.004774,0.042850,-0.011538,-0.255897,0.241002,0.140254,0.107382,-0.344080,0.319856,-0.185150,0.050404,0.341530,-0.051968,0.237719,0.124360,-0.000997,-0.073583,-0.199148,-0.053224,-0.126670,-0.005842,-0.299230,0.296949,-0.037357,0.031697,0.091046,0.168405,0.286225,0.078536,0.263840,0.188747,0.056113,0.007903,0.374924,0.221299,0.161776,-0.127480,0.105754,-0.031620,True
2,-0.134462,0.253044,0.041290,0.080743,0.053588,-0.362703,0.073922,0.733257,-0.272003,-0.259690,-0.137188,-0.480222,-0.008870,0.182584,0.091987,-0.180521,0.111676,-0.256055,-0.031775,-0.644913,0.194646,0.158881,0.089974,-0.238877,-0.059527,0.040374,-0.341575,-0.135872,-0.260633,0.087794,0.241260,-0.011441,0.134366,-0.088765,-0.150098,0.279130,0.097360,-0.258300,-0.211780,-0.545704,0.033375,-0.235535,-0.086438,0.028839,0.307675,-0.170513,-0.135141,-0.056684,0.150748,0.064947,0.142616,-0.206611,-0.001596,-0.081485,-0.030698,0.073371,0.221275,-0.108854,-0.290772,0.126918,-0.036330,0.014540,0.050204,-0.010140,-0.311745,0.300758,0.192485,0.147718,-0.407058,0.383958,-0.204763,0.035026,0.425656,-0.048118,0.278466,0.111676,-0.021807,-0.095062,-0.229167,-0.067758,-0.133468,0.024283,-0.314189,0.372889,0.012658,0.054938,0.129167,0.181531,0.391166,0.091822,0.361442,0.252149,0.056664,0.062673,0.432143,0.260767,0.191473,-0.133985,0.091730,-0.015607,False
3,-0.109986,0.319041,0.013460,0.091999,0.048233,-0.470771,0.090374,0.908940,-0.396872,-0.285608,-0.150484,-0.621303,0.019225,0.227041,0.147179,-0.255671,0.154608,-0.289697,-0.052208,-0.827037,0.275147,0.124350,0.130671,-0.295393,-0.115990,0.003202,-0.381685,-0.208303,-0.304687,0.140537,0.394790,-0.040573,0.192122,-0.166500,-0.181173,0.402168,0.134967,-0.367205,-0.326594,-0.679763,0.056695,-0.278717,-0.141231,0.012855,0.333476,-0.215075,-0.168285,-0.150031,0.193257,0.063506,0.194929,-0.279013,0.016530,-0.004745,-0.079164,0.058414,0.209117,-0.156847,-0.383591,0.102354,-0.014476,0.000519,0.064139,-0.023825,-0.412322,0.376637,0.213193,0.160999,-0.547795,0.488083,-0.291491,0.095671,0.515340,-0.084387,0.360959,0.216115,0.024445,-0.120463,-0.320413,-0.074403,-0.204968,-0.031394,-0.477954,0.445324,-0.083487,0.051057,0.128540,0.247082,0

In [64]:
# df['Output'] = y

In [65]:
# df.head()

In [66]:
df['Output'].info()

<class 'pandas.core.series.Series'>
Index: 5558 entries, 0 to 5568
Series name: Output
Non-Null Count  Dtype
--------------  -----
5558 non-null   bool 
dtypes: bool(1)
memory usage: 48.8 KB


In [67]:
y = df['Output']

In [68]:
# y = y.to_numpy()

In [69]:
# !pip install imblearn

In [70]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)

X_resample, y_resample = smote.fit_resample(X,y)

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X_resample, y_resample , test_size=0.2)

In [72]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
5367,-0.074360,0.242079,0.021030,0.076793,0.028583,-0.359966,0.086425,0.713792,-0.306501,-0.228689,-0.101508,-0.475569,0.007653,0.177186,0.109498,-0.202601,0.115761,-0.222398,-0.030309,-0.637129,0.212345,0.107814,0.106321,-0.242547,-0.093704,0.005554,-0.293326,-0.158994,-0.246775,0.098646,0.293412,-0.028030,0.164788,-0.151154,-0.140315,0.315017,0.108599,-0.269169,-0.269597,-0.533197,0.026554,-0.213368,-0.100866,0.014463,0.286586,-0.164908,-0.132902,-0.110186,0.159157,0.070519,0.149055,-0.206623,-0.008290,-0.042574,-0.043443,0.038981,0.179847,-0.122953,-0.282183,0.074891,-0.020815,0.007231,0.055186,-0.029884,-0.325883,0.300631,0.166908,0.138776,-0.442602,0.370618,-0.214788,0.063599,0.407002,-0.042554,0.287754,0.152110,0.023054,-0.094909,-0.253714,-0.047880,-0.161356,-0.023870,-0.369427,0.355412,-0.051415,0.027795,0.109429,0.187905,0.355439,0.107617,0.276920,0.242694,0.070268,0.017857,0.455289,0.265533,0.212746,-0.135927,0.126526,-0.067666
3945,-0.086534,0.236400,0.011690,0.071940,0.043867,-0.353859,0.069616,0.683843,-0.297450,-0.216344,-0.118325,-0.463889,0.012310,0.169100,0.110296,-0.189784,0.112874,-0.224423,-0.037187,-0.620597,0.203754,0.108192,0.100519,-0.230719,-0.083417,0.005017,-0.292397,-0.160138,-0.235166,0.100792,0.286731,-0.025605,0.143992,-0.121837,-0.141504,0.299948,0.098428,-0.262632,-0.236968,-0.513121,0.044652,-0.211750,-0.101561,0.015441,0.253420,-0.162578,-0.129822,-0.100643,0.143446,0.047106,0.142611,-0.207991,0.013310,-0.018383,-0.054191,0.047838,0.164246,-0.119432,-0.279130,0.086051,-0.011399,0.004046,0.050913,-0.017678,-0.307011,0.283360,0.163482,0.123494,-0.417498,0.365209,-0.217239,0.063119,0.387083,-0.056301,0.265536,0.149338,0.012873,-0.089607,-0.235933,-0.057660,-0.146857,-0.014828,-0.348414,0.338441,-0.057091,0.038592,0.101418,0.183581,0.330514,0.087922,0.281760,0.222019,0.064636,0.011441,0.434782,0.252920,0.189582,-0.144799,0.131104,-0.048538
2074,-0.075510,0.273497,0.007229,0.085426,0.036733,-0.404253,0.090451,0.779563,-0.362816,-0.237257,-0.125195,-0.537960,0.024900,0.195760,0.135755,-0.234986,0.127293,-0.238501,-0.042553,-0.719387,0.238849,0.095794,0.130641,-0.269480,-0.106496,-0.005374,-0.325054,-0.191698,-0.263337,0.121170,0.347953,-0.042227,0.182835,-0.166829,-0.155927,0.350609,0.121092,-0.296967,-0.291702,-0.589584,0.040845,-0.231019,-0.121363,0.017600,0.294819,-0.179685,-0.141450,-0.141611,0.169046,0.064985,0.164936,-0.238528,0.007568,-0.009130,-0.059751,0.041864,0.187083,-0.136033,-0.321085,0.080385,-0.008576,0.002911,0.065025,-0.022367,-0.361697,0.332860,0.180091,0.138237,-0.492587,0.414306,-0.253833,0.076304,0.446525,-0.058882,0.313453,0.184292,0.027320,-0.099797,-0.280717,-0.055899,-0.179266,-0.036330,-0.423952,0.381125,-0.085553,0.029880,0.116164,0.211101,0.376211,0.113226,0.294847,0.259996,0.070114,-0.005298,0.518275,0.291072,0.223102,-0.167489,0.161909,-0.072130
4403,-0.079345,0.258624,0.013125,0.061450,0.039032,-0.378992,0.073947,0.713867,-0.314961,-0.232153,-0.118635,-0.491587,0.015765,0.184058,0.116300,-0.199438,0.118663,-0.223888,-0.045275,-0.664999,0.215284,0.098136,0.110049,-0.243840,-0.097639,0.003619,-0.307422,-0.171993,-0.247490,0.111859,0.312064,-0.035246,0.161775,-0.140679,-0.141081,0.314933,0.108774,-0.278747,-0.260067,-0.547824,0.038634,-0.218348,-0.118177,0.012509,0.274718,-0.167805,-0.131988,-0.119482,0.158134,0.057354,0.151532,-0.219841,0.007109,-0.010875,-0.056648,0.053971,0.169852,-0.119775,-0.298500,0.084307,-0.008638,0.000069,0.052342,-0.009084,-0.324571,0.300175,0.176091,0.122821,-0.431266,0.386937,-0.235142,0.071375,0.410713,-0.059124,0.286309,0.170301,0.013954,-0.097963,-0.252945,-0.063818,-0.152956,-0.025518,-0.384810,0.362453,-0.067825,0.036542,0.108212,0.194430,0.340688,0.1

In [73]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7700 entries, 5367 to 7323
Data columns (total 100 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       7700 non-null   float64
 1   1       7700 non-null   float64
 2   2       7700 non-null   float64
 3   3       7700 non-null   float64
 4   4       7700 non-null   float64
 5   5       7700 non-null   float64
 6   6       7700 non-null   float64
 7   7       7700 non-null   float64
 8   8       7700 non-null   float64
 9   9       7700 non-null   float64
 10  10      7700 non-null   float64
 11  11      7700 non-null   float64
 12  12      7700 non-null   float64
 13  13      7700 non-null   float64
 14  14      7700 non-null   float64
 15  15      7700 non-null   float64
 16  16      7700 non-null   float64
 17  17      7700 non-null   float64
 18  18      7700 non-null   float64
 19  19      7700 non-null   float64
 20  20      7700 non-null   float64
 21  21      7700 non-null   float64
 22  2

In [74]:
y_train.head()

5367     True
3945     True
2074     True
4403     True
7271    False
Name: Output, dtype: bool

In [75]:
y_train.count()

7700

In [76]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1926 entries, 7247 to 4096
Data columns (total 100 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1926 non-null   float64
 1   1       1926 non-null   float64
 2   2       1926 non-null   float64
 3   3       1926 non-null   float64
 4   4       1926 non-null   float64
 5   5       1926 non-null   float64
 6   6       1926 non-null   float64
 7   7       1926 non-null   float64
 8   8       1926 non-null   float64
 9   9       1926 non-null   float64
 10  10      1926 non-null   float64
 11  11      1926 non-null   float64
 12  12      1926 non-null   float64
 13  13      1926 non-null   float64
 14  14      1926 non-null   float64
 15  15      1926 non-null   float64
 16  16      1926 non-null   float64
 17  17      1926 non-null   float64
 18  18      1926 non-null   float64
 19  19      1926 non-null   float64
 20  20      1926 non-null   float64
 21  21      1926 non-null   float64
 22  2

In [77]:
from sklearn.ensemble import RandomForestClassifier
# classifier1 = RandomForestClassifier()

In [78]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
rf_cv = RandomizedSearchCV(rf, param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
rf_cv.fit(X_train,y_train)

print("Best parameters:", rf_cv.best_params_)
classifier1 = rf_cv.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 30}


In [79]:
y_pred1 = classifier1.predict(X_test)

In [80]:
# df['Output'].value_counts()

In [81]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,precision_score, recall_score, f1_score

In [82]:
print(accuracy_score(y_test, y_pred1))

0.9766355140186916


In [83]:
cm = confusion_matrix(y_test, y_pred1)

In [84]:
cm

array([[950,  27],
       [ 18, 931]], dtype=int64)

In [85]:
ps= precision_score(y_test, y_pred1)
ps

0.9718162839248434

In [86]:
rs= recall_score(y_test, y_pred1)
rs

0.9810326659641728

In [87]:
f1s= f1_score(y_test, y_pred1)
f1s

0.9764027267960147